In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 

df = pd.read_csv('Company_Rating_Data.csv')

In [3]:
# choose relevant columns 
df.columns

Index(['Company Name', 'Number of Reviews', 'Average Salary', 'Job Openings',
       'Location Website', 'Global Size', 'Industry', 'Description',
       'Company Rating', 'Company Webpage', 'Headquarters', 'Global Size 2',
       'Company Location', 'Company Ownership Type', 'Year Founded',
       'Est Yearly Revenue', 'Industry 2'],
      dtype='object')

In [4]:
# Check for and remove records with default values ('Unknown', -1)
df = df[df['Global Size'] != -1] # 883 additional
df = df[df['Global Size'] != 'Unknown']
df = df[df['Company Rating'] != -1] # 144 additional
df = df[df['Company Ownership Type'] != 'Unknown'] # 27 additional/
df = df[df['Company Ownership Type'] != -1] # 27 additional
df = df[df['Year Founded'] != -1] # 1,053 additional
df = df[df['Est Yearly Revenue'] != 'Unknown / Non-Applicable'] # 1,430 additional
df = df[df['Est Yearly Revenue'] != -1] # 1,430 additional

In [1]:
# from https://github.com/jordanwheeler7/capstone-crop-yield/blob/main/crop_yield.ipynb
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import plotly.express as px
import statsmodels.api as sm
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from IPython.display import display
from math import sqrt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.inspection import permutation_importance

In [ ]:
# Define the features and target variable
features = ['Global Size','Est Yearly Revenue','Year Founded']
X = df_yield[features]
y = df_yield['Company Rating']

# Create a column transformer to apply OneHotEncoder to the 'Item' column
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Item'])
    ],
    remainder='passthrough'
)

# Create a pipeline with the preprocessor and the LinearRegression model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Fit the pipeline to the data
pipeline.fit(X, y)

# Get the feature names after one-hot encoding
ohe = preprocessor.named_transformers_['onehot']
feature_names = ohe.get_feature_names_out(input_features=['Item'])
other_features = ['avg_temp', 'Pesticides', 'avg_precipitation']
all_features = list(feature_names) + other_features

# Get the coefficients for the features from the model
coefficients = pipeline.named_steps['model'].coef_

# Print out the features with their coefficients
feature_coefficients = dict(zip(all_features, coefficients))
for feature, coef in feature_coefficients.items():
    print(f'{feature}: {coef}')

# Print out the intercept
intercept = pipeline.named_steps['model'].intercept_
print(f'Intercept: {intercept}')

In [ ]:
# Define the features and target variable
categorical_features = ['Item']
numerical_features = ['avg_temp', 'Pesticides', 'avg_precipitation']
features = numerical_features + categorical_features
X = df_yield[features]
y = df_yield['Yield']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a column transformer to apply OneHotEncoder to the 'Item' column
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

# Define polynomial features with interaction only
poly_features = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

# Create a pipeline with the preprocessor, polynomial features, and a linear regression model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('polynomial_features', poly_features),
    ('linear_regression', LinearRegression())
])

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Predict and evaluate on training data
y_train_pred = pipeline.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train =  np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Predict and evaluate on testing data
y_test_pred = pipeline.predict(X_test)
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test =  np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results for training data
print(f'Training Data - R-squared: {r2_train}, Mean Squared Error: {mse_train}, Root Mean Squared Error: {rmse_train}, Mean Absolute Error: {mae_train}')

# Print the results for testing data
print(f'Testing Data - R-squared: {r2_test}, Mean Squared Error: {mse_test}, Root Mean Squared Error: {rmse_test}, Mean Absolute Error: {mae_test}')

In [ ]:
# Encoding Categorical Features
categorical_features = ['Global Size', 'Est Yearly Revenue']
numerical_features = ['Year Founded']


preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

X = df_rating[categorical_features + numerical_features]
y = df_rating['Company Rating']

# # Splitting data
# X_transformed = preprocessor.fit_transform(X)
# X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# std_scaler = StandardScaler(with_mean=False)
# X_train_scale = std_scaler.fit_transform(X_train)
# # Scale test data for evaluation
# X_test_scale = std_scaler.transform(X_test)

In [ ]:
# https://github.com/jordanwheeler7/capstone-crop-yield/blob/main/crop_yield.ipynb
# Random Forest Best Parameters for my data: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 300}
rf = RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=300, random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

# R-squared
rf_train_score = r2_score(y_train, y_train_pred)
rf_test_score = r2_score(y_test, y_test_pred)

# Mean Absolute Error (MAE)
rf_train_mae = mean_absolute_error(y_train, y_train_pred)
rf_test_mae = mean_absolute_error(y_test, y_test_pred)

# Mean Squared Error (MSE)
rf_train_mse = mean_squared_error(y_train, y_train_pred)
rf_test_mse = mean_squared_error(y_test, y_test_pred)

# Root Mean Squared Error (RMSE)
rf_train_rmse = np.sqrt(rf_train_mse)
rf_test_rmse = np.sqrt(rf_test_mse)

# Printing the results
print(f"Random Forest - Training R-squared: {rf_train_score}, Testing R-squared: {rf_test_score}")
print(f"Random Forest - Training MAE: {rf_train_mae}, Testing MAE: {rf_test_mae}")
print(f"Random Forest - Training MSE: {rf_train_mse}, Testing MSE: {rf_test_mse}")
print(f"Random Forest - Training RMSE: {rf_train_rmse}, Testing RMSE: {rf_test_rmse}")

In [ ]:
# https://github.com/jordanwheeler7/capstone-crop-yield/blob/main/crop_yield.ipynb
# Gradient Boosting Best Parameters for my data: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300}
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb.fit(X_train, y_train)

# Predictions
y_train_pred = gb.predict(X_train)
y_test_pred = gb.predict(X_test)

# R-squared
gb_train_score = r2_score(y_train, y_train_pred)
gb_test_score = r2_score(y_test, y_test_pred)

# Mean Absolute Error (MAE)
gb_train_mae = mean_absolute_error(y_train, y_train_pred)
gb_test_mae = mean_absolute_error(y_test, y_test_pred)

# Mean Squared Error (MSE)
gb_train_mse = mean_squared_error(y_train, y_train_pred)
gb_test_mse = mean_squared_error(y_test, y_test_pred)

# Root Mean Squared Error (RMSE)
gb_train_rmse = np.sqrt(gb_train_mse)
gb_test_rmse = np.sqrt(gb_test_mse)

# Printing the results
print(f"Gradient Boosting - Training R-squared: {gb_train_score}, Testing R-squared: {gb_test_score}")
print(f"Gradient Boosting - Training MAE: {gb_train_mae}, Testing MAE: {gb_test_mae}")
print(f"Gradient Boosting - Training MSE: {gb_train_mse}, Testing MSE: {gb_test_mse}")
print(f"Gradient Boosting - Training RMSE: {gb_train_rmse}, Testing RMSE: {gb_test_rmse}")

In [ ]:
# https://github.com/jordanwheeler7/capstone-crop-yield/blob/main/crop_yield.ipynb
# Decision Tree Best Parameters for my data: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
dt = DecisionTreeRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=10, random_state=42)
dt.fit(X_train, y_train)

# Predictions
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

# R-squared
dt_train_score = r2_score(y_train, y_train_pred)
dt_test_score = r2_score(y_test, y_test_pred)

# Mean Absolute Error (MAE)
dt_train_mae = mean_absolute_error(y_train, y_train_pred)
dt_test_mae = mean_absolute_error(y_test, y_test_pred)

# Mean Squared Error (MSE)
dt_train_mse = mean_squared_error(y_train, y_train_pred)
dt_test_mse = mean_squared_error(y_test, y_test_pred)

# Root Mean Squared Error (RMSE)
dt_train_rmse = np.sqrt(dt_train_mse)
dt_test_rmse = np.sqrt(dt_test_mse)

# Printing the results
print(f"Decision Tree - Training R-squared: {dt_train_score}, Testing R-squared: {dt_test_score}")
print(f"Decision Tree - Training MAE: {dt_train_mae}, Testing MAE: {dt_test_mae}")
print(f"Decision Tree - Training MSE: {dt_train_mse}, Testing MSE: {dt_test_mse}")
print(f"Decision Tree - Training RMSE: {dt_train_rmse}, Testing RMSE: {dt_test_rmse}")

In [ ]:
# https://github.com/jordanwheeler7/capstone-crop-yield/blob/main/crop_yield.ipynb
# K-Nearest Neighbors Best Parameters: {'algorithm': 'auto', 'n_neighbors': 7, 'weights': 'uniform'}
knn = KNeighborsRegressor(algorithm= 'auto', n_neighbors=7, weights= 'distance')
knn.fit(X_train, y_train)

# Predictions
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

# R-squared
knn_train_score = r2_score(y_train, y_train_pred)
knn_test_score = r2_score(y_test, y_test_pred)

# Mean Absolute Error (MAE)
knn_train_mae = mean_absolute_error(y_train, y_train_pred)
knn_test_mae = mean_absolute_error(y_test, y_test_pred)

# Mean Squared Error (MSE)
knn_train_mse = mean_squared_error(y_train, y_train_pred)
knn_test_mse = mean_squared_error(y_test, y_test_pred)

# Root Mean Squared Error (RMSE)
knn_train_rmse = np.sqrt(knn_train_mse)
knn_test_rmse = np.sqrt(knn_test_mse)

# Printing the results
print(f"K-Nearest Neighbors - Training R-squared: {knn_train_score}, Testing R-squared: {knn_test_score}")
print(f"K-Nearest Neighbors - Training MAE: {knn_train_mae}, Testing MAE: {knn_test_mae}")
print(f"K-Nearest Neighbors - Training MSE: {knn_train_mse}, Testing MSE: {knn_test_mse}")
print(f"K-Nearest Neighbors - Training RMSE: {knn_train_rmse}, Testing RMSE: {knn_test_rmse}")

In [ ]:
# https://github.com/jordanwheeler7/capstone-crop-yield/blob/main/crop_yield.ipynb
# Buidling the Neural Network Model
nn_model = MLPRegressor(hidden_layer_sizes=(250), max_iter=1000, random_state=42)

# Training the model
nn_model.fit(X_train, y_train)

# Predicting and evaluating the model
y_train_pred = nn_model.predict(X_train)
y_test_pred = nn_model.predict(X_test)

# R-squared
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Mean Absolute Error (MAE)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# Mean Squared Error (MSE)
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Root Mean Squared Error (RMSE)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)

# Printing the results
print(f"Neural Network - Training R-squared: {train_r2}, Testing R-squared: {test_r2}")
print(f"Neural Network - Training MAE: {train_mae}, Testing MAE: {test_mae}")
print(f"Neural Network - Training MSE: {train_mse}, Testing MSE: {test_mse}")
print(f"Neural Network - Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")

In [ ]:
# https://github.com/jordanwheeler7/capstone-crop-yield/blob/main/crop_yield.ipynb
"""

To utilize this block, uncomment def perform_grid_search.
The code is commented out due to the extinsive time and
resources required to run the GridSearch.

"""

def perform_grid_search(estimator, param_grid, model_name, X_train, y_train, X_test, y_test):
    grid_search = GridSearchCV(estimator, param_grid, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, y_train)

    best_estimator = grid_search.best_estimator_
    predictions = best_estimator.predict(X_test)

    best_mae = -grid_search.best_score_  # neg_mean_absolute_error is returned negative by GridSearchCV
    mse = mean_squared_error(y_test, predictions)
    rmse = sqrt(mse)
    r2 = r2_score(y_test, predictions)

    print(f"{model_name} Best Parameters: {grid_search.best_params_}")
    print(f"{model_name} Best MAE: {best_mae}")
    print(f"{model_name} Best MSE: {mse}")
    print(f"{model_name} Best RMSE: {rmse}")
    print(f"{model_name} Best R-squared: {r2}")


# Random Forest Grid Search
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
perform_grid_search(RandomForestRegressor(random_state=42), rf_params, 'Random Forest', X_train, y_train, X_test, y_test)

# Gradient Boosting Grid Search
gb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 10]
}
perform_grid_search(GradientBoostingRegressor(random_state=42), gb_params, 'Gradient Boosting', X_train, y_train, X_test, y_test)

# Decision Tree Grid Search
dt_params = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
perform_grid_search(DecisionTreeRegressor(random_state=42), dt_params, 'Decision Tree', X_train, y_train, X_test, y_test)

# K-Nearest Neighbors Grid Search
knn_params = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}
perform_grid_search(KNeighborsRegressor(), knn_params, 'K-Nearest Neighbors', X_train, y_train, X_test, y_test)

# # Neural Network Grid Search
# nn_params = {
#     'hidden_layer_sizes': [(50,), (100,), (50,50), (100,100)],
#     'activation': ['identity', 'logistic', 'tanh', 'relu'],
#     'solver': ['lbfgs', 'sgd', 'adam'],
#     'alpha': [0.0001, 0.001, 0.01, 0.1],
#     'learning_rate': ['constant', 'invscaling', 'adaptive'],
#     'max_iter': [200, 500, 1000]
# }

# Perform grid search for MLPRegressor
perform_grid_search(MLPRegressor(random_state=42), nn_params, 'Neural Network', X_train, y_train, X_test, y_test)

In [ ]:
# Bring in One Hot Encoder Features
ohe_feature_names = preprocessor.named_transformers_['onehot'].get_feature_names_out(categorical_features)

# Combine the one-hot encoded feature names with the numerical feature names
full_feature_names = np.concatenate((ohe_feature_names, numerical_features))

# Ensure the length of `full_feature_names` matches the number of features in `feature_importance`
assert len(full_feature_names) == len(feature_importance), "Mismatch in the number of features and importances."

# Sort the feature importances and get the indices for the top features
sorted_indices = np.argsort(feature_importance)[::-1]
top_n = min(len(full_feature_names), 10)
sorted_indices_top = sorted_indices[:top_n]

# Plot the feature importances
plt.figure(figsize=(12, 8))
plt.barh(range(top_n), feature_importance[sorted_indices_top], align='center')
plt.yticks(range(top_n), full_feature_names[sorted_indices_top], fontsize=12) 
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Top Feature Importances for Gradient Boosting Model')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()